  文件名：multiple_linear_regression.ipynb
  
  作者：森森
  
  
  描述：多元线性回归作业
  
  时间：2021-10-12

# 导入包

In [69]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.linear_model import LinearRegression

# 读取数据

In [70]:
df = pd.read_csv('..\\source\\house_prices.csv')
df.info()#显示列名和数据类型类型
df.head(6)#显示前n行，n默认为5

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6028 entries, 0 to 6027
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   house_id      6028 non-null   int64 
 1   neighborhood  6028 non-null   object
 2   area          6028 non-null   int64 
 3   bedrooms      6028 non-null   int64 
 4   bathrooms     6028 non-null   int64 
 5   style         6028 non-null   object
 6   price         6028 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 329.8+ KB


,house_id,neighborhood,area,bedrooms,bathrooms,style,price
0,1112,B,1188,3,2,ranch,598291
1,491,B,3512,5,3,victorian,1744259
2,5952,B,1134,3,2,ranch,571669
3,3525,A,1940,4,2,ranch,493675
4,5108,B,2208,6,4,victorian,1101539
5,7507,C,1785,4,2,lodge,455235


# 异常值处理函数

In [71]:
# 异常值处理
# ================ 异常值检验函数：iqr & z分数 两种方法 =========================
def outlier_test(data, column, method=None, z=2):
    """ 以某列为依据，使用 上下截断点法 检测异常值(索引) """
    """ 
    full_data: 完整数据
    column: full_data 中的指定行，格式 'x' 带引号
    return 可选; outlier: 异常值数据框 
    upper: 上截断点;  lower: 下截断点
    method：检验异常值的方法（可选, 默认的 None 为上下截断点法），
            选 Z 方法时，Z 默认为 2
    """
    # ================== 上下截断点法检验异常值 ==============================
    if method == None:
        print(f'以 {column} 列为依据，使用 上下截断点法(iqr) 检测异常值...')
        print('=' * 70)
        # 四分位点；这里调用函数会存在异常
        column_iqr = np.quantile(data[column], 0.75) - np.quantile(data[column], 0.25)
        # 1，3 分位数
        (q1, q3) = np.quantile(data[column], 0.25), np.quantile(data[column], 0.75)
        # 计算上下截断点
        upper, lower = (q3 + 1.5 * column_iqr), (q1 - 1.5 * column_iqr)
        # 检测异常值
        outlier = data[(data[column] <= lower) | (data[column] >= upper)]
        print(f'第一分位数: {q1}, 第三分位数：{q3}, 四分位极差：{column_iqr}')
        print(f"上截断点：{upper}, 下截断点：{lower}")
        return outlier, upper, lower
    # ===================== Z 分数检验异常值 ==========================
    if method == 'z':
        """ 以某列为依据，传入数据与希望分段的 z 分数点，返回异常值索引与所在数据框 """
        """ 
        params
        data: 完整数据
        column: 指定的检测列
        z: Z分位数, 默认为2，根据 z分数-正态曲线表，可知取左右两端的 2%，
           根据您 z 分数的正负设置。也可以任意更改，知道任意顶端百分比的数据集合
        """
        print(f'以 {column} 列为依据，使用 Z 分数法，z 分位数取 {z} 来检测异常值...')
        print('=' * 70)
        # 计算两个 Z 分数的数值点
        mean, std = np.mean(data[column]), np.std(data[column])
        upper, lower = (mean + z * std), (mean - z * std)
        print(f"取 {z} 个 Z分数：大于 {upper} 或小于 {lower} 的即可被视为异常值。")
        print('=' * 70)
        # 检测异常值
        outlier = data[(data[column] <= lower) | (data[column] >= upper)]
        return outlier, upper, lower

# 获得异常数据并丢弃

In [72]:
outlier, upper, lower = outlier_test(data=df, column='price', method='z')#获得异常数据
outlier.info(); outlier.sample(5)
df.drop(index=outlier.index, inplace=True)#丢弃异常数据

以 price 列为依据，使用 Z 分数法，z 分位数取 2 来检测异常值...
取 2 个 Z分数：大于 1801467.1287622033 或小于 -293051.3610117055 的即可被视为异常值。
<class 'pandas.core.frame.DataFrame'>
Int64Index: 335 entries, 22 to 6018
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   house_id      335 non-null    int64 
 1   neighborhood  335 non-null    object
 2   area          335 non-null    int64 
 3   bedrooms      335 non-null    int64 
 4   bathrooms     335 non-null    int64 
 5   style         335 non-null    object
 6   price         335 non-null    int64 
dtypes: int64(5), object(2)
memory usage: 20.9+ KB


# 获取非数值类型每种值的个数

In [59]:
# 获取名义变量中各个取值的个数
# 类别变量，又称为名义变量，nominal variables
nominal_vars = ['neighborhood', 'style']

for each in nominal_vars:
    print(each, ':')
    print(df[each].agg(['value_counts']).T)
    # 直接 .value_counts().T 无法实现下面的效果
     ## 必须得 agg，而且里面的中括号 [] 也不能少
    print('='*35)

neighborhood :
                 B     A     C
value_counts  2427  1876  1725
style :
              victorian  ranch  lodge
value_counts       2997   1802   1229


# 对两列名义变量进行虚拟变量设置

In [39]:
# 对名义变量neighborhood进行处理
# 设置虚拟变量
nominal_data = df['neighborhood']
# 设置虚拟变量
dummies = pd.get_dummies(nominal_data)
dummies.sample() # pandas 会自动帮你命名
# 每个名义变量生成的虚拟变量中，需要各丢弃一个，这里以丢弃C为例
dummies.drop(columns=['C'], inplace=True)
dummies.sample()

,A,B
1578,0,0


In [40]:
# 对名义变量style进行处理
# 设置虚拟变量
nominal_style_data = df['style']
# 设置虚拟变量
style_dummies = pd.get_dummies(nominal_style_data)
style_dummies.sample() # pandas 会自动帮你命名
# 每个名义变量生成的虚拟变量中，需要各丢弃一个，这里以丢弃lodge
#原因：转化后的虚拟变量需要舍弃一个，才能得到满秩矩阵，可理解为当变量名可划分为n类时，只需要n-1个虚拟变量就能获取所有信息了
style_dummies.drop(columns=['lodge'], inplace=True)
style_dummies.sample()


,ranch,victorian
3051,1,0


# 数据拼接

In [41]:
#数据拼接
results = pd.concat(objs=[df, dummies], axis='columns') # 按照列来合并
results = pd.concat(objs=[results, style_dummies], axis='columns') # 按照列来合并
results.sample(3)

,house_id,neighborhood,area,bedrooms,bathrooms,style,price,A,B,ranch,victorian
5636,5202,B,2121,4,2,victorian,1058344,0,1,0,1
784,1901,B,2488,4,2,ranch,1239275,0,1,1,0
2164,1107,A,2837,6,4,victorian,716435,1,0,0,1


# 不加入非数值变量

In [82]:
#取出自变量和因变量
data_x=df[['area','bedrooms','bathrooms']]
data_y=df['price']

# 加入虚拟变量

In [80]:
#取出自变量和因变量
data_x=results[['area','bedrooms','bathrooms','A','B','ranch','victorian']]
data_y=results['price']

# 进行多元线性回归

In [81]:
# 进行多元线性回归
model=LinearRegression()
l_model=model.fit(data_x,data_y)
print('参数权重')
print(model.coef_)
print('模型截距')
print(model.intercept_)


参数权重
[ 2.74433353e+02  2.94082369e+04 -1.45068464e+04  9.70587414e+02
  4.58549361e+05  1.76678529e+04  3.21298434e+04]
模型截距
-143381.30896764516
